# Deep Averageing Netrok Analysis

## Comparing Rsults
||**Normal Dan 50 Embed** | **Pretrained Glove Emed 50** |
|--|-----|-------------------|
|**Test Accuracy** | 0.96087 | 0.97809 |
|**Avg Test Loss** | .0031 | .00051|

### Noramal Dan 50 Emeddings
![normal](normal_50d_plot.png)


### Pretrained Glove 50 Emeddings
![normal](glove_50d_plot.png)

In [1]:
import torch

embd = torch.nn.Embedding(20, 5)
x = torch.tensor([[1, 2, 3], [0, 2, 4]], dtype=torch.int32)
print(x)
out = embd(x)
print(out)
print(out.shape)
print(out.sum(axis=1))
print(out.sum(axis=1).shape)
lenthes = torch.tensor([3, 2])
print(lenthes.shape)
out.sum(axis=1)/lenthes.reshape([-1, 1])

/home/abdullah/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[1, 2, 3],
        [0, 2, 4]], dtype=torch.int32)
tensor([[[-1.3553,  0.3069,  0.2670, -0.2410,  0.2717],
         [-0.4535, -2.4088,  0.4335,  0.1645,  0.1725],
         [ 0.9605, -0.3291,  1.4922, -0.3915, -1.0452]],

        [[ 0.0897,  0.6092, -1.0941, -0.1198,  0.1071],
         [-0.4535, -2.4088,  0.4335,  0.1645,  0.1725],
         [ 0.3787, -0.8685,  0.8409, -0.4836, -0.5041]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])
tensor([[-0.8484, -2.4311,  2.1927, -0.4679, -0.6009],
        [ 0.0149, -2.6681,  0.1804, -0.4388, -0.2246]], grad_fn=<SumBackward1>)
torch.Size([2, 5])
torch.Size([2])


tensor([[-0.2828, -0.8104,  0.7309, -0.1560, -0.2003],
        [ 0.0074, -1.3341,  0.0902, -0.2194, -0.1123]], grad_fn=<DivBackward0>)

In [2]:
out = torch.randn(10, 2)
labels = torch.randint(0, 2,size=(10,))
labels
topval, topi = out.topk(1, dim=1)
true_vals = labels[labels == topi.reshape([10,])].shape[0]
error_vals = torch.nonzero(labels-topi.squeeze()).shape[0]
print('accuracy=', true_vals/labels.shape[0])
print('accuracy=', 1-(error_vals/labels.shape[0]))

accuracy= 0.6
accuracy= 0.6


# Glove Emeddings
### Loading Glove Resources
* [main link](https://medium.com/mlearning-ai/load-pre-trained-glove-embeddings-in-torch-nn-embedding-layer-in-under-2-minutes-f5af8f57416a)
* [second link](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76)
* [Download Page](https://nlp.stanford.edu/projects/glove/)

In [3]:
# Unkown token in i_to_word[-1] <unk>
glove_path = '/home/abdullah/Downloads/glove.6B/'
glove_type = 'glove.6B.50d.txt'
file_name = glove_path + glove_type
i_to_word,embeddings = [],[]
words = set()
word_to_i = {}
with open(file_name,'rt') as fi:
    full_content = fi.read().strip().split('\n')
    print(full_content[-1])
for i in range(len(full_content)):
    word = full_content[i].split(' ')[0]
    embed = [float(val) for val in full_content[i].split(' ')[1:]]
    i_to_word.append(word)
    word_to_i[word] = i
    words.add(word)
    embeddings.append(embed)

# adding padding token:
kPAD = '<pad>'
i_to_word.append(kPAD)
word_to_i[kPAD] = len(word_to_i)
words.add(kPAD)
embeddings.append(len(embeddings[0]) * [0.0]) 
embeddings[word_to_i[kPAD]][:10]

<unk> 0.072617 -0.51393 0.4728 -0.52202 -0.35534 0.34629 0.23211 0.23096 0.26694 0.41028 0.28031 0.14107 -0.30212 -0.21095 -0.10875 -0.33659 -0.46313 -0.40999 0.32764 0.47401 -0.43449 0.19959 -0.55808 -0.34077 0.078477 0.62823 0.17161 -0.34454 -0.2066 0.1323 -1.8076 -0.38851 0.37654 -0.50422 -0.012446 0.046182 0.70028 -0.010573 -0.83629 -0.24698 0.6888 -0.17986 -0.066569 -0.48044 -0.55946 -0.27594 0.056072 -0.18907 -0.59021 0.55559


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [4]:
# adding pytorch layer
embeddings_tensor = torch.tensor(embeddings)
embed_layer = torch.nn.Embedding.from_pretrained(embeddings_tensor,
                                                padding_idx=word_to_i[kPAD])
print(embed_layer.weight.shape)
print(embed_layer.embedding_dim)

torch.Size([400002, 50])
50


In [5]:
test_w = 'fool'
print(embeddings_tensor[word_to_i[test_w]][:6].numpy())
w_vec =  torch.tensor([ word_to_i[test_w]])
print(embed_layer(w_vec)[0][:6].numpy())

[-0.075706 -0.16436   0.18032  -0.6262    0.52178  -0.14113 ]
[-0.075706 -0.16436   0.18032  -0.6262    0.52178  -0.14113 ]


# Training

## Training with no pretrained Embeddings

In [6]:
!python3  dan.py --num-epochs=100 --checkpoint=10 

Number of Classes= 2
DanModel(
  (embeddings): Embedding(12678, 50, padding_idx=0)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
Saving Model ............
# of steps=10, Avg Train Loss=0.005377, Avg Dev Loss=0.005619, Train Acc=0.962358, Dev Acc=0.646865, Time: 1.929317
Saving Model ............
# of steps=20, Avg Train Loss=0.005284, Avg Dev Loss=0.005535, Train Acc=0.969531, Dev Acc=0.696370, Time: 2.103139
----------------------------------------------------------

start epoch 1
Saving Model ............
# of steps=10, Avg Train Loss=0.005159, Avg Dev Loss=0.005373, Train Acc=0.965909, Dev Acc=0.699670, Time: 0.318959
Saving Model ............
# of steps=

## Traing with glove Embeddings

In [7]:
!python3  dan.py --num-epochs=100 --checkpoint=10 \
    --use-glove --glove-weights='/home/abdullah/Downloads/glove.6B/glove.6B.50d.txt' \
    #--resume --load-model='glove_50d_q_type.pt'  

Loading Glove Embeddings.......
Number of Classes= 2
DanModel(
  (embeddings): Embedding(400002, 50, padding_idx=400001)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
Saving Model ............
# of steps=10, Avg Train Loss=0.005447, Avg Dev Loss=0.005336, Train Acc=0.968040, Dev Acc=0.896040, Time: 0.786209
Saving Model ............
# of steps=20, Avg Train Loss=0.004915, Avg Dev Loss=0.004992, Train Acc=0.995313, Dev Acc=0.924092, Time: 1.590576
----------------------------------------------------------

start epoch 1
Saving Model ............
# of steps=10, Avg Train Loss=0.004598, Avg Dev Loss=0.004428, Train Acc=0.980114, Dev Acc=0.925743, Time: 0.762698

In [8]:
!python3 dan.py  --help

usage: dan.py [-h] [--no-cuda] [--train-file TRAIN_FILE] [--dev-file DEV_FILE]
              [--test-file TEST_FILE] [--batch-size BATCH_SIZE]
              [--num-epochs NUM_EPOCHS] [--grad-clipping GRAD_CLIPPING]
              [--resume] [--test] [--save-model SAVE_MODEL]
              [--load-model LOAD_MODEL] [--limit LIMIT]
              [--checkpoint CHECKPOINT] [--num-workers NUM_WORKERS]
              [--use-glove] [--glove-weights GLOVE_WEIGHTS]

Question Type

options:
  -h, --help            show this help message and exit
  --no-cuda
  --train-file TRAIN_FILE
  --dev-file DEV_FILE
  --test-file TEST_FILE
  --batch-size BATCH_SIZE
  --num-epochs NUM_EPOCHS
  --grad-clipping GRAD_CLIPPING
  --resume
  --test
  --save-model SAVE_MODEL
  --load-model LOAD_MODEL
  --limit LIMIT         Number of training documents
  --checkpoint CHECKPOINT
  --num-workers NUM_WORKERS
                        Number of workers
  --use-glove           Wather to use glove or not
  --glove-weights GL